# Laboratory Exercise: Acoustic Sensors

## Milestone Items
The following are required to complete the milestone for this exercise:
- Part 1
    1. Submit to Canvas all three figures you generate using your three lists of values for speed of sound
    2. Submit to Canvas the python code used to create the three figures
- Part 2
    1. Submit to Canvas your figure of the bathymetry along the dock
    2. Submit to Canvas the python code used to create your figure
    
Provide a 2-3 sentence response addressing the questions proposed on the milestone.

# Part 1: Determining the Speed of Sound in Air

For this activity, you will be using a **HCSR04** 40 kHz ultrasonic sensor to determine the speed of sound in air.  This sensor sends out a pulse of sound, then listens for the echo return.  Based on the delay between sending the signal and receiving an echo back, you will be able to determine how far away an object is from the sensor.

In this instance we are unsure of the speed of sound in air.  By collecting measurements at a known distance, we can see how changing the speed of sound affects the accuracy of distance the sensor measures, and narrow in on the correct value.  

## Assembling the HCSR04 Sensor
Begin by connecting your **HCSR04** to your **ESP8266**.  The HCSR04 requires 5 volts, so for this excersize you will use your microcontroller in a breadboard, powered by the USB cable from your computer.  The USB cable provides 5V to the board, which means we can use the *USB* pin on the ESP8266 to give 5V to the HCSR04.

The HCSR04 sensor has 4 pins, *GND*, *VCC*, *Trig*, and *Echo*.  For this excercise, connect the *trig* pin on the HCSR04 to GPIO \#12 on the ESP8266, and the *echo* pin to GPIO \#14.  Connect *GND* to *GND* and *5V* to *USB*.

## Estimating c

We are now going to record a series of measurements at a known distance while adjusting sound speed (`c`) to determine the correct value.  

Let's import the packages we'll need to work through the rest of the lab activity:

In [ ]:
import serial # We will need the serial library to communicate with the sensor via USB
from time import sleep # The 'sleep' function will tell python to wait to hear back from the board
import matplotlib.pyplot as plt # We will use matplotlib to plot the data
%matplotlib inline # This tells jupyter to show us the plot right in the notebook

We're using a package called `pyserial` to talk to the microcontroller via USB.  **You will need to set the name of your COM port below.**  If you don't know it, you can check by using BeagleTerm. Edit then run the following cell:

In [ ]:
ser = serial.Serial('COM18',baudrate=115200) # The first argument is the com port of your sensor

Now that we can talk to our sensor through this notebook, we'll start by trying to import the library we need to run the sensor.  The driver for this sensor is already included in the firmware on your **ESP8266** as a module named `hcsr04`.  Read through then run the following cell:

In [17]:
ser.write(b'import hcsr04 \r') # send 'import hcsr04' to the board
sleep(1) # wait 1 second before checking for a response
input_data = ser.read(ser.inWaiting()) # read the response
if input_data.find(b'Error') != -1: # If the word 'Error' appears in the response
    print('Import Failed, check serial command or libraries') # print fail message
else: # if no 'Error'
    print('Import Successful') # success!

Import Successful


The `hcsr04.HCSR04` class requires three input variables:
- `trigger_pin`,  the GPIO pin on the ESP8266 corresponding to the **Trig** pin on the HCSR04
- `echo_pin`,  the GPIO pin on the ESP8266 corresponding to the **Echo** pin on the HCSR04
- `c`, the speed of sound to use to calculate distance

We're going to pass a command to the board that tells it how to use the acoustic sensor using the following code:

`sensor = hcsr04.HCSR04(trigger_pin = 12, echo_pin = 14, c = 300)`

### Step 1 - Define the arguments
We can set up the arguments separately so they're easy to edit when we want to change them:

In [ ]:
triggerPin = 12 #board pin number to 'Trig' on sensor
echoPin = 14 #board pin number to 'Echo' on sensor
soundSpeed = range(300,400,10) # a list of sound speeds, i.e., can be range() or [200]

### Step 2 - Collecting measurements
Now we'll send the setup to the board, and tell it to take a measurement and report back a distance for each value of sound speed we ask.  After defining the sound speed, we will use `sensor.distance()` to take a measurement.

Your ESP8266 should return a float value that corresponds to the distance of the object in front of the sensor in **millimeters**.  Each line from the board will be echoed so we can see the results, and these numbers will get stored in a list.

In [ ]:
distance = []
c = []
for speed in soundSpeed:
    command = 'sensor = hcsr04.HCSR04(trigger_pin = '+str(triggerPin)+', echo_pin = '+str(echoPin)+', c = '+str(speed)+')\r'
    ser.write(bytes(command, 'utf-8'))
    ser.write(b'sensor.distance()\r')
    sleep(.5)
    input_data = ser.read(ser.inWaiting())
    print('Command Sent: '+ str(input_data).split('\\r\\n>>')[0])
    print('Distance measured: ',float(str(input_data).split('\\n')[2][:-2]))
    distance.append(float(str(input_data).split('\\n')[2][:-2]))
    c.append(float(str(input_data).split('\\n')[0].split('c = ')[-1][:-3]))

### Step 3 - Plot the measurements

Fill in the missing information below in order to plot how distance measured changes as a function of sound speed:

In [ ]:
setDistance = 246 # The reference distance
plt.plot(c,distance)
plt.xlabel('c')
plt.ylabel('distance (mm)')
plt.axhline(y=setDistance,linestyle = '--',color = 'g')

Based on the intercept of the dashed line (your known distance) and the solid line (the measured distance at sound speed *c*), create a new list of values for `c` at a finer resolution (intervals of 5?  intervals of 3?) that will help you zoom in on the `c` value of that intercept.

Repeat **Step 1 - Step 3**, each time changing the values used for sound speed, and determine a new set of vales for `distance`. 

Try increasing the resolution of  `c` one more time, trying values with an interval of 1.  Again, define a new set of values for `c` on your ESP8266 and determine a new set of vales for `distance`.  Create a third figure with the data.

Based on the data and figures, what do you estimate the speed of sound in air to be?  What do you notice about the variability in the distance measurements as you increase the resolution of your `c` list?  What would you do to improve accuracy of your measurement?

Now that you have a more accurate value for the speed of sound, test the limits of your sensor by adjusting the values in **Step 1** and running the code in **Step 2**.  Does there appear to be a minimum distance you can measure?  Does there appear to be a maximum distance?  What happens when it seems like you might be too far away or too close to take a measurement?

# Part 2: Mapping the Seafloor

For this activity, you will be using a **HCSR04** 40 kHz ultrasonic sensor to determine the bathymetry along the MSB dock.  The HCSR04 sensor sends out a pulse of sound through one transducer, and listens for the echo return in the second transducer.  Based on the delay between sending the signal and receiving an echo back, you will be able to determine how far away an object is from the sensor.

This activity uses the waterproof transducer version of the HCSR04 ultrasonic acoustic sensor to create a rough bathymetric chart of a region of off a pier or dock.  For this excercise you will collect measurements of the depth to the seafloor along the Marine Sciences Building dock with 1 meter spacing.

## Bathymetry Measurements
You will be using the HCSR04 to collect measurements of depth at a series of fixed positions along the dock.  Using the tape measure, start at the western corner of the dock and take a measurement at 1 meter spacing.  Lower the transducer into the water to the tape mark indicating 10cm is at the surface.  Collect and record samples in the following format:

|Distance on Dock (m) | Depth (mm)|
| :---:  |  :---: |
|1| Meaurement \#1,  \#2,  \#3,  \#4,  \#5|
|2| Meaurement \#1,  \#2,  \#3,  \#4,  \#5|
|...|...|
|*n*| Meaurement \#1,  \#2,  \#3,  \#4,  \#5|

Collect repeat samples at each fixed distance point.  Do all of the values collected appear to be valid?

Once your sample table is complete, create a plot of the bathymetry in *Canopy* using the Python code below.  For the depth value, take the median of the measurements at each distance that you believe to be correct in order to remove bad samples or noise.  

In [65]:
def dist(triggerPin,echoPin,soundSpeed,site,allSamples):
    triggerPin = 12 #board pin number to 'Trig' on sensor
    echoPin = 14 #board pin number to 'Echo' on sensor
    for speed in soundSpeed:
        command = 'sensor = hcsr04.HCSR04(trigger_pin = '+str(triggerPin)+', echo_pin = '+str(echoPin)+', c = '+str(speed)+')\r'
        ser.write(bytes(command, 'utf-8'))
        ser.write(b'sensor.distance()\r')
        sleep(.5)
        input_data = ser.read(ser.inWaiting())
        print('Command Sent: '+ str(input_data).split('\\r\\n>>')[0])
        print('Distance measured: ',float(str(input_data).split('\\n')[2][:-2]))
        if site in allSamples:
            allSamples[site].append(float(str(input_data).split('\\n')[2][:-2]))
        else:
            allSamples[site] = [float(str(input_data).split('\\n')[2][:-2])]
        return allSamples

In [66]:
soundSpeed = [300] # a list of sound speeds, i.e., can be range() or [200]
allSamples = {} # let's make an empty dictionary that we'll fill

In [67]:
site = 5
allSamples = dist(triggerPin,echoPin,soundSpeed,site,allSamples)
allSamples

Command Sent: b'sensor = hcsr04.HCSR04(trigger_pin = 12, echo_pin = 14, c = 300)
Distance measured:  220.0


{5: [220.0]}

In [70]:
allSamples.items()

dict_items([(5, [220.0])])